# Cayley-Dickson Construction Applied to Qi Definition

*Version 1*

My original implementation of Gaussian integers included two classes, ``Zi`` and ``Qi``, where, for example, ``Zi(2, -7)`` represents a Gaussian integer, and ``Qi(-2/3, 4/5)`` represents a Gaussian rational.

I'd like to extend this code to include rational-valued quaternions and octonions using the Cayley-Dickson construction.

The [Cayley-Dickson construction](https://en.wikipedia.org/wiki/Cayley%E2%80%93Dickson_construction) is a process by which one can use a recursive definition of conjugation together with a recursive definition of multiplication to use...
* pairs of real numbers ($\mathbb{R}$) to create complex numbers,
* pairs of complex numbers ($\mathbb{C}$) to create quaternions,
* pairs of quaternions ($\mathbb{H}$) to create octonions,
* pairs of octonions ($\mathbb{O}$) to create sedenions ($\mathbb{S}$), and so on.

For more specifics, see my write-up about the Cayley-Dickson construction [at this link](https://abstract-algebra.readthedocs.io/en/latest/55_cayley_dickson.html).

In [1]:
from cayley_dickson_alg import Zi, SetScalarMult
from random import randint

In [2]:
from random import seed

seed(42)  # Generate the same random sequence each time (for testing)

In [3]:
from fractions import Fraction
from functools import wraps

In [4]:
def gaussian_rational(fnc):
    """For use as a property that casts an argument into Gaussian rational."""
    @wraps(fnc)
    def gaussian_rational_wrapper(arg, num):
        qi = to_gaussian_rational(num)
        return fnc(arg, qi)
    return gaussian_rational_wrapper

In [ ]:
dbg = True
indent = "  "

In [22]:
def debug(indent, num, *args):
    print(f"***{indent} [{num}] {args}")

In [13]:
class Qi:
    """Gaussian Rational Number Class"""

    __MAX_DENOMINATOR = 1_000_000

    def __init__(self, re=None, im=None):

        # --( 1 )--------------------------------------------------------
        # re is a str, float, int, or Fraction; and
        # im is a str, float, int, Fraction, or None

        if isinstance(re, (str, float, int, Fraction)):
            debug(
            if isinstance(re, Fraction):
                self.__re = re
            else:
                self.__re = Fraction(re)
            if im is None:
                self.__im = Fraction(0)
            elif isinstance(im, (str, float, int, Fraction)):
                if isinstance(im, Fraction):
                    self.__im = im
                else:
                    self.__im = Fraction(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --( 2 )--------------------------------------------------------
        # re is a complex; and
        # im is complex, a Qi, or None

        elif isinstance(re, complex):
            print("------- In Clause #2 -------------")
            if im is None:
                self.__re = Fraction(re.real)
                self.__im = Fraction(re.imag)
            elif isinstance(im, (Qi, complex)):
                self.__re = Qi(re.real, re.imag)
                self.__im = Qi(im.real, im.imag)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --( 3 )--------------------------------------------------------
        # re is a Qi, and im is complex, a Qi, or None

        elif isinstance(re, Qi):
            print("------- In Clause #3 -------------")
            if im is None:
                self.__re = re.real
                self.__im = re.imag
            elif isinstance(im, (complex, Qi)):
                self.__re = Qi(re)
                self.__im = Qi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --( 3.5 )--------------------------------------------------------
        # re is a Zi, and im is a Zi, or None

        elif isinstance(re, Zi):
            print("------- In Clause #3.5 -------------")
            if im is None:
                if isinstance(re.real, Zi) and isinstance(re.imag, Zi):
                    self.__re = Qi(re.real)
                    self.__im = Qi(re.imag)
                else:
                    self.__re = re.real
                    self.__im = re.imag
            elif isinstance(im, Zi) and im.order() == re.order():
                if isinstance(re.real, Zi) and isinstance(re.imag, Zi):
                    self.__re = Qi(re)
                    self.__im = Qi(im)
                else:
                    self.__re = re.real
                    self.__im = re.imag
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --( 4 )--------------------------------------------------------
        # re is a list or tuple of numbers with length equal to a
        # power of 2, and im is None, or it is a tuple or list
        # similar to the one input for re.
        elif isinstance(re, (tuple, list)):
            print("------- In Clause #4 -------------")
            z = Zi.from_array(re)
            if im is None:
                self.__re = z.real
                self.__im = z.imag
            elif isinstance(im, (tuple, list)) and len(im) == len(re):
                w = Zi.from_array(im)
                self.__re = z
                self.__im = w
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --( 5 )--------------------------------------------------------
        # Both re and im are None

        elif re is None:
            print("------- In Clause #5 -------------")
            self.__re = 0
            if im is None:
                self.__im = 0
            else:
                raise Exception(f"If re is None, then im must be None. But im = {im}")
        else:
            raise Exception(f"Unexpected combination of input types: {re} and {im}")

    @classmethod
    def max_denominator(cls, denom=None):
        if denom is None:
            return cls.__MAX_DENOMINATOR
        elif isinstance(denom, int) and denom > 1:
            cls.__MAX_DENOMINATOR = denom
            return cls.__MAX_DENOMINATOR
        else:
            raise ValueError(f"Maximum denominator, {denom}, must be an integer > 1")

    @property
    def real(self) -> Fraction:
        return self.__re

    @property
    def imag(self) -> Fraction:
        return self.__im

    def __repr__(self):
        return f"{self.__class__.__name__}({self.__re}, {self.__im})"

#    def __str__(self):
#        if self.__im < 0:
#            return f"({self.__re}{self.__im}j)"
#        else:
#            return f"({self.__re}+{self.__im}j)"

    @gaussian_rational
    def __add__(self, other):
        return Qi(self.__re + other.real, self.__im + other.imag)

In [14]:
half = Fraction(1, 2)
two_thirds = Fraction(2, 3)
print(half, two_thirds)

1/2 2/3


In [15]:
print(f"{Qi() = }")
print(f"{Qi(1) = }")
print(f"{Qi(1.5) = }")
print(f"{Qi(two_thirds) = }")
print(f"{Qi(1, 2) = }")
print(f"{Qi(1.5, 4.25) = }")
print(f"{Qi('1/2', '2/3') = }")
print(f"{Qi(half) = }")
print(f"{Qi('1/2', '2/3') = }")
print(f"{Qi(half, two_thirds) = }")

------- In Clause #5 -------------
Qi() = Qi(0, 0)
------- In Clause #1 -------------
Qi(1) = Qi(1, 0)
------- In Clause #1 -------------
Qi(1.5) = Qi(3/2, 0)
------- In Clause #1 -------------
Qi(two_thirds) = Qi(2/3, 0)
------- In Clause #1 -------------
Qi(1, 2) = Qi(1, 2)
------- In Clause #1 -------------
Qi(1.5, 4.25) = Qi(3/2, 17/4)
------- In Clause #1 -------------
Qi('1/2', '2/3') = Qi(1/2, 2/3)
------- In Clause #1 -------------
Qi(half) = Qi(1/2, 0)
------- In Clause #1 -------------
Qi('1/2', '2/3') = Qi(1/2, 2/3)
------- In Clause #1 -------------
Qi(half, two_thirds) = Qi(1/2, 2/3)


In [16]:
print(f"{Qi((1-2j)) = }")
print(f"{Qi(Qi(1, -2)) = }")
print(f"{Qi((1-2j), (-3+5j)) = }")
print(f"{Qi((1.5-2j), (-4.25+5j)) = }")
print(f"{Qi(Qi(1, -2), (-3+5j)) = }")
print(f"{Qi((1-2j), Qi(-3, 5)) = }")

------- In Clause #2 -------------
Qi((1-2j)) = Qi(1, -2)
------- In Clause #1 -------------
------- In Clause #3 -------------
Qi(Qi(1, -2)) = Qi(1, -2)
------- In Clause #2 -------------
------- In Clause #1 -------------
------- In Clause #1 -------------
Qi((1-2j), (-3+5j)) = Qi(Qi(1, -2), Qi(-3, 5))
------- In Clause #2 -------------
------- In Clause #1 -------------
------- In Clause #1 -------------
Qi((1.5-2j), (-4.25+5j)) = Qi(Qi(3/2, -2), Qi(-17/4, 5))
------- In Clause #1 -------------
------- In Clause #3 -------------
------- In Clause #3 -------------
------- In Clause #2 -------------
Qi(Qi(1, -2), (-3+5j)) = Qi(Qi(1, -2), Qi(-3, 5))
------- In Clause #1 -------------
------- In Clause #2 -------------
------- In Clause #1 -------------
------- In Clause #1 -------------
Qi((1-2j), Qi(-3, 5)) = Qi(Qi(1, -2), Qi(-3, 5))


In [17]:
Qi((1-2j), (-3+5j))

------- In Clause #2 -------------
------- In Clause #1 -------------
------- In Clause #1 -------------


Qi(Qi(1, -2), Qi(-3, 5))

In [18]:
q0 = Qi(Qi(1, 2), Qi(3, 4))
q0

------- In Clause #1 -------------
------- In Clause #1 -------------
------- In Clause #3 -------------
------- In Clause #3 -------------
------- In Clause #3 -------------


Qi(Qi(1, 2), Qi(3, 4))

In [19]:
Qi(Zi(1, 2))

------- In Clause #3.5 -------------


Qi(1, 2)

In [20]:
Qi(Zi(1, 2), Zi(3, 4))

------- In Clause #3.5 -------------


Qi(1, 2)

In [23]:
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def calculate_average(numbers):
    logging.debug(f"Calculating average for numbers: {numbers}")
    if not numbers:
        logging.warning("Attempted to calculate average of an empty list.")
        return 0
    total = sum(numbers)
    average = total / len(numbers)
    logging.info(f"Average calculated: {average}")
    return average

calculate_average([1, 2, 3, 4, 5])
calculate_average([])

2025-10-11 14:08:17,564 - DEBUG - Calculating average for numbers: [1, 2, 3, 4, 5]
2025-10-11 14:08:17,566 - INFO - Average calculated: 3.0
2025-10-11 14:08:17,566 - DEBUG - Calculating average for numbers: []
2025-10-11 14:08:17,567 - WARNING - Attempted to calculate average of an empty list.


0

In [24]:
import logging

# Configure basic logging to a file and the console
logging.basicConfig(
    level=logging.INFO,  # Set the minimum level of messages to log
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the log message format
    handlers=[
        logging.FileHandler("app.log"),  # Log messages to a file named 'app.log'
        logging.StreamHandler()  # Also output log messages to the console
    ]
)

# Get a logger instance (it's good practice to use __name__)
logger = logging.getLogger(__name__)

# Log messages at different levels
logger.debug("This is a debug message.")  # Won't be shown with INFO level
logger.info("This is an informational message.")
logger.warning("This is a warning message.")
logger.error("This is an error message.")
logger.critical("This is a critical error message.")

def divide(a, b):
    try:
        result = a / b
        logger.info(f"Division of {a} by {b} resulted in {result}")
        return result
    except ZeroDivisionError:
        logger.error("Attempted to divide by zero!", exc_info=True) # exc_info adds traceback
        return None

divide(10, 2)
divide(5, 0)

2025-10-11 14:10:41,646 - DEBUG - This is a debug message.
2025-10-11 14:10:41,647 - INFO - This is an informational message.
2025-10-11 14:10:41,647 - WARNING - This is a warning message.
2025-10-11 14:10:41,648 - ERROR - This is an error message.
2025-10-11 14:10:41,648 - CRITICAL - This is a critical error message.
2025-10-11 14:10:41,648 - INFO - Division of 10 by 2 resulted in 5.0
2025-10-11 14:10:41,649 - ERROR - Attempted to divide by zero!
Traceback (most recent call last):
  File "/var/folders/n6/_xg9_wkd7mg3w6sxqfh7z55h0000gn/T/ipykernel_5375/3114523761.py", line 25, in divide
    result = a / b
             ~~^~~
ZeroDivisionError: division by zero
